# Аугментации

In [141]:
import Augmentor

In [142]:
def augment_images(folder, max_count=0, in_max_count=0):
    for animal in folder:
        num_of_augmented = 0
        augmentator = Augmentor.Pipeline(animal)
        if (max_count==0):
            if (in_max_count!=0):
                num_of_augmented = in_max_count * len(augmentator.augmentor_images)
            else: num_of_augmented = 0
        else: num_of_augmented = max_count
        p_flip = 0.68
        augmentator.flip_left_right(probability=p_flip)
        p_skew = 0.63
        augmentator.skew_left_right(probability=p_skew, magnitude=0.3)
        p_rotate = 0.7
        augmentator.rotate(probability=p_rotate, max_left_rotation=15, max_right_rotation=15)
        p_zoom = 0.65
        augmentator.zoom(probability=p_zoom, min_factor=1.1, max_factor=1.25)
        p_erase = 0.45
        augmentator.random_erasing(probability=p_erase,rectangle_area=0.23)
        p_bright = 0.55
        augmentator.random_brightness(probability=p_bright, min_factor=1.15, max_factor=1.3)
        p_contrast = 0.55
        augmentator.random_contrast(probability=p_contrast, min_factor=1.1, max_factor=1.25)
        # print(len(augmentator.augmentor_images))        
        augmentator.sample(num_of_augmented)

In [143]:
augment_images(['data/baldcat/', 'data/haircat/'], max_count=1000)

Initialised with 1020 image(s) found.
Output directory set to data/baldcat/output.

Processing <PIL.Image.Image image mode=RGB size=1000x667 at 0x248425ECC50>:   0%|          | 1/1000 [00:03<57:58,  3.48s/ Samples]


KeyboardInterrupt: 

In [ ]:
import shutil
import os
for file in os.listdir('data/haircat/output'):
    shutil.move('data/haircat/output/'+file, 'data/haircat/')
shutil.rmtree('data/haircat/output')
for file in os.listdir('data/baldcat/output'):
    shutil.move('data/baldcat/output/'+file, 'data/baldcat/')
shutil.rmtree('data/baldcat/output')

In [ ]:

def null_output(paths):
    for animal in paths:
        output_dir = animal + "output"
        shutil.rmtree(output_dir, ignore_errors=False, onerror=False)

# Train-val-test-split


In [ ]:
input_folder = "data"
for dir in os.listdir(input_folder):
    for file in os.listdir(input_folder+'/'+dir):
        os.rename(input_folder+'/'+dir+'/'+file, f'{dir}_image_{file}')

In [ ]:
import splitfolders
output = "sortedData"
if not os.path.exists(output):
    splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.7, .15, .15))

Copying files: 2040 files [00:05, 382.17 files/s]


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

# Инициализация нужных в дальнейшем переменных

In [3]:
train_dir = "sortedData/train"
test_dir = "sortedData/test"
val_dir = "sortedData/val"
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 256
nb_train_samples = 1418 
nb_validation_samples = 306 
nb_test_samples = 306


# Imports для СНС

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG19
from keras.optimizers import Adam

# Инициализация СНС

In [5]:
vgg19_net = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)



In [6]:
vgg19_net.trainable = False


In [7]:
vgg19_net.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
model = Sequential()
# Добавляем в модель сеть dense16_net вместо слоя
model.add(vgg19_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5), 
              metrics=['accuracy'])

# Data Generator (Определение классов из наших директорий)

In [11]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )

Found 1428 images belonging to 2 classes.


In [13]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )


Found 306 images belonging to 2 classes.


In [14]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
    )

Found 306 images belonging to 2 classes.


# Обучение

In [15]:
from PIL import Image
import sys

sys.modules['Image'] = Image

In [16]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
5/5 [==============================] - 128s 26s/step - loss: 0.7370 - accuracy: 0.5359 - val_loss: 0.6104 - val_accuracy: 0.7070
Epoch 2/5
5/5 [==============================] - 112s 23s/step - loss: 0.6006 - accuracy: 0.6698 - val_loss: 0.5156 - val_accuracy: 0.8125
Epoch 3/5
5/5 [==============================] - 107s 22s/step - loss: 0.5260 - accuracy: 0.7406 - val_loss: 0.4416 - val_accuracy: 0.9062
Epoch 4/5
5/5 [==============================] - 116s 24s/step - loss: 0.4391 - accuracy: 0.8302 - val_loss: 0.3724 - val_accuracy: 0.9219
Epoch 5/5
5/5 [==============================] - 118s 24s/step - loss: 0.3978 - accuracy: 0.8490 - val_loss: 0.3413 - val_accuracy: 0.9102


In [17]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


C:\Users\Лавандос\AppData\Local\Temp\ipykernel_12752\4219390412.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


In [18]:
print("accuracy на тестовых данных: %.2f%%" % (scores[1]*100))


accuracy на тестовых данных: 94.92%


In [19]:
vgg19_net.trainable = True
trainable = False
for layer in vgg19_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable  

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [23]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])
              

In [24]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
5/5 [==============================] - 145s 29s/step - loss: 0.3585 - accuracy: 0.8592 - val_loss: 0.2637 - val_accuracy: 0.9492
Epoch 2/2
5/5 [==============================] - 137s 30s/step - loss: 0.2357 - accuracy: 0.9275 - val_loss: 0.1764 - val_accuracy: 0.9492


In [25]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("accuracy на тестовых данных: %.2f%%" % (scores[1]*100))

C:\Users\Лавандос\AppData\Local\Temp\ipykernel_12752\1674365955.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


accuracy на тестовых данных: 96.09%
